In [ ]:
import pandas as pd
from pandas.plotting import scatter_matrix
import json
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, plot_confusion_matrix, confusion_matrix

plt.style.use('seaborn')

In [ ]:
with open('/Users/adam/phd/projects/certificates/data.json', 'r') as json_file:
    data = json.load(json_file)

### Some useful functions

In [ ]:
def plot_feature_importances(importances, feature_names):
    importances, feature_names = zip(*sorted(zip(importances, feature_names), reverse=True))
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.bar(feature_names, importances)
    plt.xticks(rotation=45)
    plt.xlabel('Feature')
    plt.ylabel('Importance')
    plt.title('Feature importance in Random Forest classifier')
    plt.show()

### Load json features into pandas dataframe

In [ ]:
# Create all features and fill a dataframe with them
feature_dict = {x: [] for x in ['category', 'scheme', 'n_updates', 'sec_level_processed', 'cert_lab', 'n_pages',
                           'cert_date', 'archived_date', 'manufacturer', 'protection_profiles', 'pdf_encrypted', 'defenses', 'crypto_algs']}
for key, val in data.items():
    cert = data[key]
    feature_dict['category'].append(cert['csv_scan'].get('cc_category', np.nan))
    feature_dict['scheme'].append(cert['csv_scan'].get('cc_scheme', np.nan))
    feature_dict['n_updates'].append(len(cert['csv_scan'].get('maintainance_updates', np.nan)))
    feature_dict['sec_level_processed'].append(cert['processed'].get('cc_security_level', np.nan))
    feature_dict['cert_lab'].append(cert['processed'].get('cert_lab', np.nan))
    feature_dict['cert_date'].append(cert['csv_scan'].get('cc_certification_date', np.nan))
    feature_dict['archived_date'].append(cert['csv_scan'].get('cc_archived_date', np.nan))
    feature_dict['manufacturer'].append(cert['processed'].get('cc_manufacturer_simple', np.nan))
    feature_dict['protection_profiles'].append(cert['csv_scan'].get('cc_protection_profiles', np.nan))

    keywords_scan = cert.get('keywords_scan') or {}
    feature_dict['defenses'].append(keywords_scan.get('rules_defenses', np.nan))
    feature_dict['crypto_algs'].append(keywords_scan.get('rules_crypto_algs', np.nan))
    
    
    meta_scan = cert.get('pdfmeta_scan') or {}
    n_pages = meta_scan.get('pdf_number_of_pages', np.nan)
    pdf_encrypted = meta_scan.get('pdf_is_encrypted', np.nan)
    feature_dict['n_pages'].append(n_pages)
    feature_dict['pdf_encrypted'].append(pdf_encrypted)
df = pd.DataFrame(data=feature_dict)

## Feature description

| Feature name | Feature type | N. unique val. | N. missing | notes |
| --- | --- | --- | --- | --- |
| category | nominal | 15 | 0 | - |
| scheme | nominal | 17 | 0 | 8 prevalent, merge below 100? |
| n_updates | ordinal | 12 | 0 | set intervals [0,0), (1,1], (2,2], (3, inf) |
| sec_level_processed | ordinal | 14 | 0 | + stands for additional defenses |
| cert_lab | nominal | 13 | 2740 | Could be very useful, need to get more |
| n_pages | numerical | - | 923 | Split into categories by histogram | 
| cert_date | Date | - | 0 | |
| archived_date | Date | - | 0 | Beware of huge number of archived in Sep. 2019|
| manufacturer | nominal | 807 | 10 | Need to merge if it should be of some use |
| protection_profiles | nominal | 216 | 0 (or 2300)| Take length of list as feature |
| pdf_encrypted | boolean | True/False | 674 | If missing set false |
| defenses | nominal | - | 668 | To be processed to number of keys |
| crypto_algs | nominal | - | 668 | To be processed to number of algs. |


In [ ]:
# Show number of missing values per each column
df.isnull().sum()

In [ ]:
## Preprocess features

### Fill missing values

In [ ]:
df.pdf_encrypted.fillna(False, inplace=True)
df.defenses = df.defenses.map(lambda x: {} if pd.isnull(x) else x)
df.crypto_algs = df.crypto_algs.map(lambda x: {} if pd.isnull(x) else x)

n_pages_median = df.n_pages.median()
df.n_pages.fillna(n_pages_median, inplace=True)

## Categorize nominal and ordinal features

In [ ]:
# map defenses and crypto algs to the length of the list
map_to_len = lambda x: len(x) if isinstance(x, dict) else 0
df['n_defenses'] = df.defenses.map(map_to_len)
df['n_crypto_algs'] = df.crypto_algs.map(map_to_len)

# map protection profiles to the length of the list
prot_prof_to_len = lambda x: len(x.split(',')) if x != '' else 0
df['n_protection_profiles'] = df.protection_profiles.map(prot_prof_to_len)

# Map security level into ordered categories
sec_level_dict = {'None': np.nan, 'EAL1': 0, 'EAL1+': 1, 'EAL2': 2, 'EAL2+': 3, 'EAL3': 4, 'EAL3+': 4, 'EAL4': 5, 'EAL4+': 6, 'EAL5': 7, 'EAL5+': 8, 'EAL6': 9, 'EAL6+': 10, 'EAL7': 11, 'EAL7+': 12}
df['sec_level_cat'] = df.sec_level_processed.map(sec_level_dict)
sec_level_median = df['sec_level_cat'].loc[df.sec_level_cat != -1].median()
df['sec_level_cat'].fillna(sec_level_median, inplace=True)

# categorize n_pages into intervals (ordered)
df['n_pages_cat'] = pd.cut(df.n_pages, [0, 10, 15, 30, 40, 50, 5000], labels=[x for x in range(6)])
df.n_pages_cat = df['n_pages_cat'].astype('int64')

# Categories number of updates to intervals
df['n_updates_cat'] = pd.cut(df.n_updates, [-1, 0, 1, 2, 3, 100], labels=[x for x in range(5)])
df['n_updates_cat'] = df['n_updates_cat'].astype('int64')

# Create year of certification feature
df.cert_date = pd.to_datetime(df.cert_date)
df.archived_date = pd.to_datetime(df.archived_date)
df['cert_year'] = df.cert_date.dt.year

# Categorize schemes, not sure if wise
n_occurences = df.scheme.value_counts()
schemes_to_merge = list(n_occurences.loc[n_occurences < 100].index)
df['scheme_cat'] = df.scheme.map(lambda x: x if x not in schemes_to_merge else 'Other')

### Basic visualisations

In [ ]:
# Show histogram
%matplotlib inline
df.hist(bins=20, figsize=(20, 15))
plt.show()

In [ ]:
# Show correlations between numerical features and the target feature
df.corr()['sec_level_cat'].sort_values(ascending=False)

In [ ]:
df.columns

### Prepare training and test set

In [ ]:
features = df.loc[:, ['category', 'n_updates', 'pdf_encrypted',
                      'n_defenses', 'n_crypto_algs', 'n_protection_profiles', 'sec_level_cat',
                      'n_pages_cat', 'n_updates_cat', 'cert_year', 'scheme_cat']]

features.category = features.category.astype('category').cat.codes
features.scheme_cat = features.scheme_cat.astype('category').cat.codes

labels = np.array(features['sec_level_cat'])
label_names = list(sec_level_dict.keys())[1:]
features.drop('sec_level_cat', axis=1, inplace=True)
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2, random_state=42, stratify=labels)
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

### Build RandomForest classifier / regressor

In [ ]:
rf = RandomForestClassifier(n_estimators = 100)
rf.fit(train_features, train_labels)
y_pred = rf.predict(test_features)

### Show results

In [ ]:
acc = accuracy_score(predictions, test_labels, normalize=True)
print(f'Accuracy: {acc}')

In [ ]:
importances = rf.feature_importances_
feature_names = list(features.columns)
plot_feature_importances(importances, feature_names)

In [ ]:
plt = plot_confusion_matrix(rf, x_test, y_test, normalize=None, cmap='Greens', xticks_rotation=45, display_labels=label_names)